In [1]:
import cv2
import numpy as np
from skimage.filters import threshold_local
import tensorflow as tf
from skimage import measure
import imutils

In [2]:
 def extract_contours(after_preprocess):
    contours, _ = cv2.findContours(after_preprocess,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    cv2.imshow('original_video', img)
    print("contours extracted")
    return contours

In [3]:
min_area = 4500  # minimum area of the plate
max_area = 30000  # maximum area of the plate
element_structure = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(22, 3))

In [4]:
def preprocess(input_img):
    #cv2.imshow("input_img",input_img)
    imgBlurred = cv2.GaussianBlur(input_img,(5,5), 0)  # old window was (5,5)
    gray = cv2.cvtColor(imgBlurred, cv2.COLOR_BGR2GRAY)  # convert to gray
    sobelx = cv2.Sobel(gray, cv2.CV_8U, 1, 0, ksize=3)  # sobelX to get the vertical edges
    ret2, threshold_img = cv2.threshold(sobelx, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    element = element_structure
    morph_n_thresholded_img = threshold_img.copy()
    cv2.morphologyEx(src=threshold_img, op=cv2.MORPH_CLOSE, kernel=element, dst=morph_n_thresholded_img)
    print("pre-processing done")
    return morph_n_thresholded_img
   

In [5]:
def sort_cont(character_contours):
    """
    To sort contours from left to right
    """
    i = 0
    boundingBoxes = [cv2.boundingRect(c) for c in character_contours]
    (character_contours, boundingBoxes) = zip(*sorted(zip(character_contours, boundingBoxes),key=lambda b: b[1][i], reverse=False))
    return character_contours
    print("sorting done")

In [6]:
def segment_chars(plate_img, fixed_width):
    """
    extract Value channel from the HSV format of image and apply adaptive thresholding
    to reveal the characters on the license plate
    """
    V = cv2.split(cv2.cvtColor(plate_img, cv2.COLOR_BGR2HSV))[2]

    T = threshold_local(V, 29, offset=15, method='gaussian')

    thresh = (V > T).astype('uint8') * 255

    thresh = cv2.bitwise_not(thresh)

    # resize the license plate region to a canoncial size
    plate_img = imutils.resize(plate_img, width=fixed_width)
    thresh = imutils.resize(thresh, width=fixed_width)
    bgr_thresh = cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR)

    # perform a connected components analysis and initialize the mask to store the locations
    # of the character candidates
    labels = measure.label(thresh,background=0,connectivity =2)

    charCandidates = np.zeros(thresh.shape, dtype='uint8')

    # loop over the unique components
    characters = []
    for label in np.unique(labels):
        # if this is the background label, ignore it
        if label == 0:
            continue
        # otherwise, construct the label mask to display only connected components for the
        # current label, then find contours in the label mask
        labelMask = np.zeros(thresh.shape, dtype='uint8')
        labelMask[labels == label] = 255

        cnts = cv2.findContours(labelMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[1] if imutils.is_cv2() else cnts[0]

        # ensure at least one contour was found in the mask
        if len(cnts) > 0:

            # grab the largest contour which corresponds to the component in the mask, then
            # grab the bounding box for the contour
            c = max(cnts, key=cv2.contourArea)
            (boxX, boxY, boxW, boxH) = cv2.boundingRect(c)

            # compute the aspect ratio, solodity, and height ration for the component
            aspectRatio = boxW / float(boxH)
            solidity = cv2.contourArea(c) / float(boxW * boxH)
            heightRatio = boxH / float(plate_img.shape[0])

            # determine if the aspect ratio, solidity, and height of the contour pass
            # the rules tests
            keepAspectRatio = aspectRatio < 1.0
            keepSolidity = solidity > 0.15
            keepHeight = heightRatio > 0.5 and heightRatio < 0.95

            # check to see if the component passes all the tests
            if keepAspectRatio and keepSolidity and keepHeight and boxW > 14:
                # compute the convex hull of the contour and draw it on the character
                # candidates mask
                hull = cv2.convexHull(c)

                cv2.drawContours(charCandidates, [hull], -1, 255, -1)

    contours, hier = cv2.findContours(charCandidates, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        contours = sort_cont(contours)
        addPixel = 4  # value to be added to each dimension of the character
        for c in contours:
            (x, y, w, h) = cv2.boundingRect(c)
            if y > addPixel:
                y = y - addPixel
            else:
                y = 0
            if x > addPixel:
                x = x - addPixel
            else:
                x = 0
            temp = bgr_thresh[y:y + h + (addPixel * 2), x:x + w + (addPixel * 2)]

            characters.append(temp)
        return characters
    else:
        return None

    print("segmenting done")


In [7]:
def find_characters_on_plate(plate):
    charactersFound = segment_chars(plate, 400)
    if charactersFound:
        print(len(charactersFound))
        return charactersFound
    print("characters found on plate and are the following")

In [8]:
 # PLATE FEATURES
def ratioCheck(area, width, height):
    min = min_area
    max = max_area

    ratioMin = 3
    ratioMax = 6

    ratio = float(width) / float(height)
    if ratio < 1:
        ratio = 1 / ratio

    if (area < min or area > max) or (ratio < ratioMin or ratio > ratioMax):
        return False
    return True

    print("ratio checked")

In [9]:
def preRatioCheck(area, width, height):
    min = min_area
    max = max_area

    ratioMin = 2.5
    ratioMax = 7

    ratio = float(width) / float(height)
    if ratio < 1:
        ratio = 1 / ratio

    if (area < min or area > max) or (ratio < ratioMin or ratio > ratioMax):
        return False
    return True

    print("pre ratio checked")

In [10]:

def validateRatio(rect):
    (x, y), (width, height), rect_angle = rect

    if (width > height):
        angle = -rect_angle
    else:
        angle = 90 + rect_angle

    if angle > 15:
        return False
    if (height == 0 or width == 0):
        return False

    area = width * height
    if not preRatioCheck(area, width, height):
        return False
    else:
        return True

    print("ratio validated")

In [11]:
def clean_plate(plate):
    gray = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    if contours:
        areas = [cv2.contourArea(c) for c in contours]
        max_index = np.argmax(areas)  # index of the largest contour in the area array

        max_cnt = contours[max_index]
        max_cntArea = areas[max_index]
        x, y, w, h = cv2.boundingRect(max_cnt)
        rect = cv2.minAreaRect(max_cnt)
        rotatedPlate = plate
        if not ratioCheck(max_cntArea, rotatedPlate.shape[1], rotatedPlate.shape[0]):
            return plate, False, None
        return rotatedPlate, True, [x, y, w, h]
    else:
        return plate, False, None
    print("plate cleaned")


In [12]:
import matplotlib.pyplot as plt
def check_plate(input_img, contour):
    min_rect = cv2.minAreaRect(contour)
    if validateRatio(min_rect):
#         print("validate ratio satisfied" + str(min_rect))
        x, y, w, h = cv2.boundingRect(contour)
        after_validation_img = input_img[y:y + h, x:x + w]
        after_clean_plate_img, plateFound, coordinates =clean_plate(after_validation_img)
        if plateFound:
            print("plate found and need to extract characters on plate-calling find_characters_on_plate ")
            characters_on_plate = find_characters_on_plate(after_clean_plate_img)
            #print("characters_on_plate: "+ str(characters_on_plate))
           
            if (characters_on_plate is not None and len(characters_on_plate) == 8):
                x1, y1, w1, h1 = coordinates
                coordinates = x1 + x, y1 + y
                after_check_plate_img = after_clean_plate_img
                print("plate checked")
                return after_check_plate_img, characters_on_plate, coordinates
    return None, None, None
    

In [13]:
char_on_plate = []
def find_possible_plates(input_img):
    """
    Finding all possible contours that can be plates
    """
    plates = []
    corresponding_area = []

    after_preprocess = preprocess(input_img)
    possible_plate_contours = extract_contours(after_preprocess)
#     print("Possible plate contours: "+str(possible_plate_contours))
    print("contour extraction completed and need to find characters- calling check_plate")

    for cnts in possible_plate_contours:
        plate, characters_on_plate, coordinates = check_plate(input_img, cnts)
        if plate is not None:
            plates.append(plate)
            char_on_plate.append(characters_on_plate)
            corresponding_area.append(coordinates)
            print("plate is not none")

    if (len(plates) > 0):
        return plates
        print("some plates found")
    else:
        return None
        print("No plates found")
    print("find possible plates executed")

In [14]:
# Neural network   
def load_graph( modelFile):
    graph = tf.Graph()
    graph_def = tf.compat.v1.GraphDef()
    with open(modelFile, "rb") as f:
        graph_def.ParseFromString(f.read())
    with graph.as_default():
        tf.import_graph_def(graph_def)
    return graph
    print("graph loaded")

In [15]:
# Neural network   
def load_label(labelFile):
    label = []
    proto_as_ascii_lines = tf.io.gfile.GFile(labelFile).readlines()
    for l in proto_as_ascii_lines:
        label.append(l.rstrip())
    return label
    print("label_loaded")

In [16]:
# Neural network   
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
model_file = "./model/binary_128_0.50_ver3.pb"
label_file = "./model/binary_128_0.50_labels_ver2.txt"
label = load_label(label_file)
graph = load_graph(model_file)
sess = tf.compat.v1.Session(graph=graph)

def label_image(tensor):

    input_name = "import/input"
    output_name = "import/final_result"

    input_operation = graph.get_operation_by_name(input_name)
    output_operation = graph.get_operation_by_name(output_name)

    results = sess.run(output_operation.outputs[0],{input_operation.outputs[0]: tensor})
    results = np.squeeze(results)
    labels = label
    top = results.argsort()[-1:][::-1]
    return labels[top[0]]

Instructions for updating:
non-resource variables are not supported in the long term


In [17]:
# Neural network  
def convert_tensor(image, imageSizeOuput):
    """
takes an image and tranform it in tensor
"""
    image = cv2.resize(image, dsize=(imageSizeOuput, imageSizeOuput), interpolation=cv2.INTER_CUBIC)
    np_image_data = np.asarray(image)
    np_image_data = cv2.normalize(np_image_data.astype('float'), None, -0.5, .5, cv2.NORM_MINMAX)
    np_final = np.expand_dims(np_image_data, axis=0)
    return np_final
    print("image tensor")

In [18]:
# Neural network   


def label_image_list(listImages, imageSizeOuput):
    #function to call neural network
    plate = ""
    for img in listImages:
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
        plate = plate +label_image(convert_tensor(img, imageSizeOuput))
    print("plate_output" + str(plate))
    return plate, len(plate)


In [19]:
import matplotlib.pyplot as plt
def find_plates(img):
    possible_plates = find_possible_plates(img)
    plt.imshow(img)
    if possible_plates is not None:
        for i, p in enumerate(possible_plates):
            chars_on_plate = char_on_plate[i]
            recognized_plate, _ = label_image_list(chars_on_plate, imageSizeOuput=128)
            print(recognized_plate)
            cv2.imshow('plate', p)
    print("find plates is called")

In [ ]:
import matplotlib.pyplot as plt

cap = cv2.VideoCapture('video.mp4')
if (cap.isOpened()== False):
    print("Error opening video stream or file")
while (cap.isOpened()):
    ret, input_img = cap.read()
#     print('Read a new frame: ',ret)
    if ret == True:
        cv2.imshow('original_video', input_img)
        ret, input_img = cap.read()
        cv2.imshow('original_video', input_img)
        cv2.imwrite("image_captured.png", input_img)
        plt.imshow(input_img)
        
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break
        img = input_img
        find_plates(img)  
        print("image captured")
    else:
            break
   
    
cap.release()
cv2.destroyAllWindows()

pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
plate found and need to extract characters on plate-calling find_characters_on_plate 
characters found on plate and are the following
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
plate found and need to extract characters on plate-calling find_characters_on_plate 
characters found on plate and are the following

1
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
plate found and need to extract characters on plate-calling find_characters_on_plate 
1
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
plate found and need to extract characters on plate-calling find_characters_on_plate 
characters found on plate and are the following
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
plate found and need to extract characters on plate-calling find_characters_on_plate 
characters found on plate and are the following
find plates is called
imag

pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
plate found and need to extract characters on plate-calling find_characters_on_plate 
1
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
ima

pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
plate found and need to extract characters on plate-calling find_characters_on_plate 
characters found on plate a

pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
plate found and need to extract characters on plate-calling find_characters_on_plate 
1
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
plate found and need to extract characters on plate-calling find_characters_on_plate 
characters found on plate and are the following
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
plate found and need to extract characters on plate-calling find_characters_on_plate 
1
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and ne

pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need

pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need

characters found on plate and are the following
plate found and need to extract characters on plate-calling find_characters_on_plate 
1
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
plate found and need to extract characters on plate-calling find_characters_on_plate 
2
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
plate found and need to extract characters on plate-calling find_characters_on_plate 
characters found on plate and are the following
find plates is called
image captured
pre-processing done
contours extracted
contour extraction completed and need to find characters- calling check_plate
plate found and need to ex

KeyboardInterrupt: 